# 1. Towards feature engineering

## 1.1 Take a look at public metadata

### 1.1.a Grab videos from training set

In [ ]:
import json

def read_annotations(subset='train'):
    with open(f'../data/raw/{subset}_data.json', 'r') as f:
        data = json.load(f)
    return data

def read_hash(filename='/home/escorciav/datasets/didemo/annotations/yfcc100m_hash.txt'):
    "Parse text-file with hash"
    lines = open(filename).readlines()
    yfcc100m_hash = {}
    for line_count, line in enumerate(lines):
        line = line.strip().split('\t')
        yfcc100m_hash[line[0]] = line[1]
    return yfcc100m_hash

In [ ]:
data = read_annotations('train')

Are there missing videos?

In [ ]:
import os
from pathlib import Path

video_dir = Path('/home/escorciav/datasets/didemo/videos')
video_dir = Path('/home/escorciav/Downloads/didemo/missing_videos/')

num_videos = 0
for i in data:
#     print(i.keys())
#     print(i['video'])
    video_file = video_dir / (i['video'] + '.mp4')
    if video_file.is_file():
        num_videos += 1
    else:
        video_file = 
        
print(f'[{num_videos}/{len(data)}] {len(data)-num_videos}')

### 1.1.b Check additional videos

In [ ]:
from pathlib import Path
import subprocess

extra_a = Path('/home/escorciav/Downloads/didemo/missing_videos/')
extra_b = Path('/home/escorciav/Downloads/didemo/missing_videos_AWS/')
repeated, indeed = 0, 0
for i in extra_b.iterdir():
    j = extra_a / i.name
    if j.is_file():
        repeated += 1
        cmd = ['diff', i, j]
        output = subprocess.run(cmd, stderr=subprocess.STDOUT, universal_newlines=True)
        if output.returncode == 0:
            indeed += 1
print(f'Potential repeteated files: {repeated}. Confirmed {indeed}')

Set with all videos of this dataset

We will duplicate '.mp4'

In [ ]:
from pathlib import Path

folders = [
    Path('/home/escorciav/datasets/didemo/missing/missing_videos/'),
    Path('/home/escorciav/datasets/didemo/missing/missing_videos_AWS/'),
    Path('/home/escorciav/datasets/didemo/videos/')
]

videos = set()
for i in folders:
    for j in i.iterdir():
        videos.add(j.name)
        if j.suffix == '.mp4':
            videos.add(j.stem)

num_missing = []
for i in ['train', 'val', 'test']:
    num_missing.append(0)
    for j in read_annotations(i):
        if j['video'] not in videos:
            num_missing[-1] += 1
            print(j['video'])
print(f'Number clips missing [{sum(num_missing)}-'
      f'{num_missing[0]}/{num_missing[1]}/{num_missing[2]}]')

Workaround for videos that where not in AWS or Lisas's website.

I turned out that I only needed two videos, thus I didn't automate it.

In [ ]:
from pathlib import Path

missing_videos = [
    '33234611@N00_3567949618_0bee41c6f7.mp4',
    '15389242@N00_4268126780_be074c803c.mov',
    '15389242@N00_4268126780_be074c803c.mov',
    '33234611@N00_3567949618_0bee41c6f7.mp4',
    '15389242@N00_4268126780_be074c803c.mov',
    '15389242@N00_4268126780_be074c803c.mov',
    '15389242@N00_4268126780_be074c803c.mov',
]
missing_videos = set(missing_videos)

def get_aws_link(h):
    return f'https://multimedia-commons.s3-us-west-2.amazonaws.com/data/videos/mp4/{h[:3]}/{h[3:6]}/{h}.mp4'

PATH = Path('/home/escorciav/datasets/didemo/missing/missing_trial/')
VIDEO_DIR = Path('/home/escorciav/datasets/didemo/videos/')

id2hash = read_hash()
for video_name in missing_videos:
    video_id = video_name.split('_')[1]
    video_hash = id2hash[video_id]

    filename_hash = PATH / (video_hash + '.mp4')
    if filename_hash.is_file():
        target = VIDEO_DIR / (video_name + '.mp4')
        filename_hash.rename(target)
    else:
        print('Manual download, sorry :p')

Merge missing videos into big folder

In [ ]:
from pathlib import Path

main_folder = [
    Path('/home/escorciav/datasets/didemo/videos/')
]

extra_folders = [
    Path('/home/escorciav/datasets/didemo/missing/missing_videos_AWS/'),
    Path('/home/escorciav/datasets/didemo/missing/missing_videos/'),
]

videos = set()
for i in main_folder:
    for j in i.iterdir():
        videos.add(j.name)
        if j.suffix == '.mp4':
            videos.add(j.stem)

num_missing = []
missing_set = set()
for i in ['train', 'val', 'test']:
    num_missing.append(0)
    for j in read_annotations(i):
        if j['video'] not in videos:
            num_missing[-1] += 1
            missing_set.add(j['video'])
print(f'Number clips missing [{sum(num_missing)}-'
      f'{num_missing[0]}/{num_missing[1]}/{num_missing[2]}]')

print('Moving data from extra folders to main_folders')
assert len(main_folder) == 1
main_folder = main_folder[0]
for i in missing_set:
    for j in extra_folders:
        full_i = (j / i)
        if full_i.is_file():
            target = main_folder / (i + '.mp4')
            full_i.rename(target)
            break

## 1.2 Getting metadata from videos

### 1.2.a Export video names

In [ ]:
import json

videos_file = '../data/raw/videos.csv'
video_lists = []
with open(videos_file, 'w') as fw:
    fw.write('video_id,subset\n')
    for subset in ['train', 'val', 'test']:
        dataset_filename = '../data/raw/{}_data.json'.format(subset)

        with open(dataset_filename) as fr:
            dataset = json.load(fr)
    
        video_lists.append(set())
        for i in dataset:
            video_lists[-1].add(i['video'])
            # sanity check videos to make sure videos are not mixed
            if len(video_lists) > 1:
                assert all([i['video'] not in j for j in video_lists[0:-1]])
    
        for i in video_lists[-1]:
            fw.write(f'{i},{subset}\n')

Check videos and update CSV-file accordingly

In [ ]:
from pathlib import Path
import pandas as pd

filename = '../data/raw/videos.csv'
dirname = Path('/home/escorciav/datasets/didemo/videos/')
df = pd.read_csv(filename)
for i, row in df.iterrows():
    filename = dirname / (row['video_id'] + '.mp4')
    if not filename.exists():
        assert False
    df.loc[i, 'video_id'] = row['video_id'] + '.mp4'
df.to_csv('../data/raw/cev.csv', index=False)
# I had to do this because I was sleeping and somthing was working well in that monda
!mv ../data/raw/cev.csv ../data/raw/videos.csv

### 1.2.b Export metadata with video-utils

```
python tools/video_info.py \
  -i ~/projects/moments-retrieval/data/raw/videos.txt \
  -o ~/projects/moments-retrieval/data/raw/videos_info.csv \
  -r ~/datasets/didemo/videos/ \
  -n 48 --verbose 5
```

## 1.2.c Merge CSV files

checking corrupted files

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('../data/raw/videos_info.csv')
corrupted = np.zeros(len(df), dtype=bool)
for i in ['duration', 'frame_rate', 'num_frames', 'width', 'height']:
    corrupted = np.logical_or(corrupted, pd.isna(df[i]))
print(f'Num corrupted [{corrupted.sum()}/{len(corrupted)}]')
print(df.loc[corrupted, 'video_name'])

TODO: merge CSV

[check this](https://stackoverflow.com/questions/39862654/pandas-concat-of-multiple-data-frames-using-only-common-columns)

In [ ]:
import numpy as np
import pandas as pd

df = pd.read_csv('../data/raw/videos_info.csv')
print(
    'Duration [max, min, mean]: ['
    f'{df.loc[:, "duration"].max()}, '
    f'{df.loc[:, "duration"].min()}, '
    f'{df.loc[:, "duration"].mean()}]')

In [ ]:
video_durations = df.loc[:, 'duration'].dropna()
duration_edges = [-1, 0 + 1e-3] + list(range(5, 40, 5)) + [df.loc[:, "duration"].max()]
videos_per_duration, _ = np.histogram(video_durations, bins=duration_edges)
assert len(video_durations) == videos_per_duration.sum()
for i, v in enumerate(videos_per_duration):
    print(f'[{duration_edges[i]},{duration_edges[i+1]}] = {v}')

## 1.2.d Check dumping frames

1. Generate files for dumping frames

In [ ]:
import json

filename = 'data/raw/train_data.json'
file_list = 'data/raw/videos_train.txt'
videos = set()
with open(filename) as f:
    data = json.load(f)
    for i in data:
        videos.add(i['video'])

with open(file_list, 'w') as fw:
    for i in videos:
        fw.write('{}\n'.format(i))

At Adobe. This is a replica of content from [1.2.a-b (this notebook)]()

1. Install video-utils

    ```bash
    git clone https://github.com/escorciav/video-utils
    conda create -y -n video-utils python=3.6 pandas joblib && conda activate video-utils && conda install -y ffmpeg -c conda-forge
    ```

    Note: I added a hot-fix because video name contains special characters.

    apply this diff to this commit `TODO`

    ```
    TODO
    ```

2. Extract frames

    rescale to 320x240 and frame-rate 5FPS

    ```bash
    conda activate video-utils
    python tools/batch_dump_frames.py -i [input-list] -o [output-dir] -s [csv-report] --filters "-vf fps=5 scale=320:240 -qscale:v 2" -r [root] -n 32
    ```

    _troubleshooting_

    running code

    ```
    ModuleNotFoundError: No module named 'okvideo'                                                                                              
                                                                                                                                                  Uncaught exception. Entering post mortem debugging                                                                                                                                                                                                                               Running 'cont' or 'step' will restart the program                                                                                           
    > []/video-utils/tools/batch_dump_frames.py(10)<module>()                                                        
    -> from okvideo.ffmpeg import dump_frames
    ```

    fix: `export PYTHONPATH=$PWD` if it's executed from project dir.

1. Verify if the frames were dumped succesfully.

In [ ]:
!wc -l data/raw/videos_val.txt
!wc -l data/raw/videos_test.txt
!wc -l data/raw/videos_train.txt
!ls /mnt/ssd/tmp/didemo/frames/ | wc -l

In [ ]:
1094+1037+8511

In [ ]:
import os
import pandas as pd

filename = '../data/raw/videos_train.txt'
dirname = '/mnt/ssd/tmp/didemo/frames/'
logfile = '../data/interim/log/train-frames-01.csv'
missing = 0
with open(filename) as f:
    for i in f:
        video_name = os.path.splitext(i.strip())[0]
        video_dir = os.path.join(dirname, video_name)
        if not os.path.isdir(video_dir):
            missing += 1
            continue
        if len(os.listdir(video_dir)) == 0:
            missing += 1
    print('Num videos without frames:', missing)
df = pd.read_csv(logfile, header=None)
missing_log = (df.loc[:, 1] == False).sum()
print('Num videos without frames:', missing_log)

 - Tar frames

Try to follow this name convention `_resolution_[fps].tar`

`tar -cf frames_320x240_5fps.tar frames/`

- Check size



In [ ]:
!du -chs /mnt/ssd/tmp/didemo/frames/